In [ ]:
import pandas as pd
import numpy as np
import scipy
import os

RQ1_DATA_FOLDER_PATH = "../experiment_data/RQ1_data/"

subfolders = [f for f in os.listdir(RQ1_DATA_FOLDER_PATH)
              if os.path.isdir(os.path.join(RQ1_DATA_FOLDER_PATH, f))]

rq_1_df = pd.DataFrame()

for folder in subfolders:
    xlsx_files = [f for f in os.listdir(
        RQ1_DATA_FOLDER_PATH+folder) if f.endswith(".xlsx")]
    for xlsx_file in xlsx_files:
        xlsx_file_path = RQ1_DATA_FOLDER_PATH + f"{folder}/{xlsx_file}"
        temp_df = pd.read_excel(xlsx_file_path)
        temp_df["participant"] = folder
        rq_1_df = pd.concat([rq_1_df, temp_df], ignore_index=True)



### Data Cleaning

In [ ]:
column_name_mapper = {
    "Tempo di completamento": "tempo_completamento", 
    "Tool usato?": "tool_used"
}

rq_1_df = rq_1_df.rename(columns=column_name_mapper)    

In [ ]:
import pandas as pd
import re

def normalize_time(x):
    if pd.isna(x):
        return None
    
    # Converti in stringa e ripulisci
    s = str(x).strip().replace('"', '').replace("'", "")
    
    # Separa con ":" invece di "."
    s = s.replace(".", ":")
    
    # Caso: HH:MM:SS
    if re.match(r"^\d{1,2}:\d{1,2}:\d{1,2}$", s):
        h, m, sec = s.split(":")
        if h == "00":
            total_seconds = int(m) * 60 + int(sec)    
        else:
            total_seconds = int(h) * 60 + int(m)
    
    # Caso: M:SS,ms → ignoro millisecondi
    elif re.match(r"^\d{1,2}:\d{1,2},\d+$", s):
        m, rest = s.split(":")
        sec = rest.split(",")[0]
        total_seconds = int(m) * 60 + int(sec)
    
    # Caso: M:SS
    elif re.match(r"^\d{1,2}:\d{1,2}$", s):
        m, sec = s.split(":")
        total_seconds = int(m) * 60 + int(sec)
    
    # Caso: solo numero (minuti)
    elif s.isdigit():
        total_seconds = int(s) * 60
    
    else:
        return "None"
    
    # Conversione in formato MM:SS
    minutes, seconds = divmod(total_seconds, 60)
    return f"{minutes:02d}:{seconds:02d}"

In [ ]:
def count_seconds(x):
    if pd.isna(x):
        return None
    time: str = x
    m, sec = time.split(":")
    m = int(m)
    sec = int(sec)
    return m*60 + sec


In [ ]:
rq_1_df.columns

In [ ]:
rq_1_df["tempo_completamento_normalized"] = rq_1_df["tempo_completamento"].apply(normalize_time)

In [ ]:
righe_na = rq_1_df[rq_1_df["tempo_completamento_normalized"].isna()]

In [ ]:
rq_1_df = rq_1_df.dropna(axis=0)

In [ ]:
rq_1_df["tempo_completamento_normalized_seconds"] = rq_1_df["tempo_completamento_normalized"].apply(count_seconds)

### Save Dataframe as csv

In [ ]:
#rq_1_df.to_csv(f"{RQ1_DATA_FOLDER_PATH}/all_data_cleaned.csv")

### Check Script 1 

In [ ]:
tool_used_df_script1 = rq_1_df[rq_1_df["tool_used"]==True]
tool_used_df_script1.describe()

In [ ]:
tool_not_used_df_script1 = rq_1_df[rq_1_df["tool_used"]==False]
tool_not_used_df_script1.describe()

In [ ]:
from scipy.stats import shapiro

shapiro(tool_used_df_script1["tempo_completamento_normalized_seconds"])

In [ ]:
shapiro(tool_not_used_df_script1["tempo_completamento_normalized_seconds"])

In [ ]:
from scipy.stats import mannwhitneyu

stat, p_value = mannwhitneyu(
    tool_used_df_script1["tempo_completamento_normalized_seconds"],
    tool_not_used_df_script1["tempo_completamento_normalized_seconds"],
    #alternative="two-sided"  # due code, per testare differenze in entrambe le direzioni
)

print("Statistic:", stat)
print("p-value:", p_value)

if p_value < 0.05:
    print("Possiamo rigettare ipotesi nulla")
else:
    print("NON Possiamo rigettare ipotesi nulla")